In [4]:
import pandas as pd
df = pd.read_csv(r'D:\ML_AI_DL\Fraud_Detection.csv')
df.head()
df.tail()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1048570,95,CASH_OUT,132557.35,C1179511630,479803.00,347245.65,C435674507,484329.37,616886.72,0,0
1048571,95,PAYMENT,9917.36,C1956161225,90545.00,80627.64,M668364942,0.00,0.00,0,0
1048572,95,PAYMENT,14140.05,C2037964975,20545.00,6404.95,M1355182933,0.00,0.00,0,0
1048573,95,PAYMENT,10020.05,C1633237354,90605.00,80584.95,M1964992463,0.00,0.00,0,0
1048574,95,PAYMENT,11450.03,C1264356443,80584.95,69134.92,M677577406,0.00,0.00,0,0


In [5]:
df.info

<bound method DataFrame.info of          step      type     amount     nameOrig  oldbalanceOrg  \
0           1   PAYMENT    9839.64  C1231006815      170136.00   
1           1   PAYMENT    1864.28  C1666544295       21249.00   
2           1  TRANSFER     181.00  C1305486145         181.00   
3           1  CASH_OUT     181.00   C840083671         181.00   
4           1   PAYMENT   11668.14  C2048537720       41554.00   
...       ...       ...        ...          ...            ...   
1048570    95  CASH_OUT  132557.35  C1179511630      479803.00   
1048571    95   PAYMENT    9917.36  C1956161225       90545.00   
1048572    95   PAYMENT   14140.05  C2037964975       20545.00   
1048573    95   PAYMENT   10020.05  C1633237354       90605.00   
1048574    95   PAYMENT   11450.03  C1264356443       80584.95   

         newbalanceOrig     nameDest  oldbalanceDest  newbalanceDest  isFraud  \
0             160296.36  M1979787155            0.00            0.00        0   
1            

In [15]:
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [12]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [16]:
df['type'].unique() #to checkk the unique values in column type

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [18]:
df.drop(['nameOrig','nameDest'], axis=1, inplace=True) #the columns are un useful to drop it

In [21]:
#to handle the categorical column-using one-hot encoding
df = pd.get_dummies(df,columns=['type'], drop_first = True)

In [25]:
# Creating a feature that can detect suspicious activity
df['errorBalanceOrig']=df['oldbalanceOrg']-df['newbalanceOrig']-df['amount']
df['errorBalanceDest']=df['oldbalanceDest']-df['newbalanceDest']-df['amount']

In [26]:
print(df['isFraud'].value_counts()) #checking the total number of fraud in the columns isfraud


isFraud
0    1047433
1       1142
Name: count, dtype: int64


In [31]:
#using SMOTE(synthetic minority oversampling technique-oversampling the minority class)
# as the given data is 99% not fraud, 1% fraud, thus it gives (99% accuracy but it is difficult to pridict the fraud case
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X = df.drop('isFraud',axis=1)
y = df['isFraud']

sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X,y)

print(y_resampled.value_counts())

isFraud
0    1047433
1    1047433
Name: count, dtype: int64


In [35]:
#as the above data are balance, we will do split and train of the data
X_train,X_test,y_train,y_test=train_test_split(X_resampled,y_resampled, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression() #selecting the model
model.fit(X_train,y_train) #fitting the model


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [42]:
#to evaluate the model
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[196753  12367]
 [ 13977 195877]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.94    209120
           1       0.94      0.93      0.94    209854

    accuracy                           0.94    418974
   macro avg       0.94      0.94      0.94    418974
weighted avg       0.94      0.94      0.94    418974



In [27]:
df[['oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']].describe() #checking wheather there are zero or missing values


,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,8.740095e+05,8.938089e+05,9.781600e+05,1.114198e+06
std,2.971751e+06,3.008271e+06,2.296780e+06,2.416593e+06
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.600200e+04,0.000000e+00,1.263772e+05,2.182604e+05
75%,1.366420e+05,1.746000e+05,9.159235e+05,1.149808e+06
max,3.890000e+07,3.890000e+07,4.210000e+07,4.220000e+07
